# Implement function calling with the Granite-3.0-8B-Instruct model in Python with watsonx

**Authors:** Erika Russi, Anna Gutowska, Jess Bozorg

In this tutorial, you will use the IBM® [Granite-3.0-8B-Instruct model](https://www.ibm.com/granite) now available on watsonx.ai™ to perform custom function calling.  

Traditional [large language models (LLMs)](https://www.ibm.com/topics/large-language-models), like the OpenAI GPT-4 (generative pre-trained transformer) model available through ChatGPT, and the IBM Granite™ models that we'll use in this tutorial, are limited in their knowledge and reasoning. They produce their responses based on the data used to train them and are difficult to adapt to personalized user queries. To obtain the missing information, these [generative AI](https://www.ibm.com/topics/generative-ai) models can integrate external tools within the function calling. This method is one way to avoid fine-tuning a foundation model for each specific use-case. The function calling examples in this tutorial will implement external [API](https://www.ibm.com/topics/api) calls. 

The Granite-3.0-8B-Instruct model and tokenizer use [natural language processing (NLP)](https://www.ibm.com/topics/natural-language-processing) to parse query syntax. In addition, the models use function descriptions and function parameters to determine the appropriate tool calls. Key information is then extracted from user queries to be passed as function arguments. 

# Steps

## Step 1. Set up your environment

While you can choose from several tools, this tutorial is best suited for a Jupyter Notebook. Jupyter Notebooks are widely used within data science to combine code with various data sources such as text, images and data visualizations. 

This tutorial walks you through how to set up an IBM account to use a Jupyter Notebook.

1. Log in to [watsonx.ai](https://dataplatform.cloud.ibm.com/registration/stepone?context=wx&apps=all) using your IBM Cloud account.

2. Create a [watsonx.ai project](https://www.ibm.com/docs/en/watsonx/saas).

	You can get your project ID from within your project. Click the Manage tab. Then, copy the project ID from the Details section of the General page. You need this ID for this tutorial.

3. Create a [Jupyter Notebook](https://www.ibm.com/docs/en/watsonx/saas?topic=editor-creating-managing-notebooks).

This step opens a notebook environment where you can copy the code from this tutorial. Alternatively, you can download this notebook to your local system and upload it to your watsonx.ai project as an asset. To view more Granite tutorials, check out the [IBM Granite Community](https://github.com/ibm-granite-community). This tutorial is also available on [Github](https://github.com/IBM/ibmdotcom-tutorials/tree/main/generative-ai/function-calling.ipynb).

To avoid Python package dependency conflicts, we recommend setting up a [virtual environment](https://docs.python.org/3/library/venv.html). 

## Step 2. Set up a Watson Machine Learning service instance and API key

1. Create a [Watson Machine Learning](https://cloud.ibm.com/catalog/services/watson-machine-learning) service instance (choose the Lite plan, which is a free instance).

2. Generate an [API Key in WML](https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/ml-authentication.html). 

3. Associate the WML service to the project you created in [watsonx.ai](https://dataplatform.cloud.ibm.com/docs/content/wsj/getting-started/assoc-services.html). 

## Step 3. Install and import relevant libraries and set up your credentials

We'll need a few libraries and modules for this tutorial. Make sure to import the following ones; if they're not installed, you can resolve this with a quick pip install.

In [1]:
# installations
%pip install transformers | tail -n 1
%pip install torch torchvision | tail -n 1
%pip install langchain-ibm | tail -n 1

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [15]:
#imports
import json
import requests
import ast
import re
import os

from transformers import AutoTokenizer
from langchain_ibm import WatsonxLLM

from dotenv import load_dotenv
load_dotenv(os.getcwd()+"/.env", override=True)

True

Next, we can prepare our environment by setting the model ID for the `granite-3-8b-instruct` model, and the tokenizer for the same Granite model. 

Replace the placeholder text with the `WATSONX_APIKEY` and `WATSONX_PROJECT_ID` you created in steps 1 and 2. 

In [16]:
MODEL_ID = "ibm/granite-3-8b-instruct"

TOKENIZER = AutoTokenizer.from_pretrained("ibm-granite/granite-3.0-8b-instruct")

WATSONX_URL = "https://us-south.ml.cloud.ibm.com"

WATSONX_APIKEY = "<YOUR_WATSONX_API_KEY_HERE>"

WATSONX_PROJECT_ID = "<YOUR_WATSONX_PROJECT_ID_HERE>"

The `get_stock_price` function in this tutorial requires an `AV_STOCK_API_KEY` key. To generate a free `AV_STOCK_API_KEY`, please visit the [Alpha Vantage website](https://www.alphavantage.co/support/#api-key). 

Secondly, the `get_current_weather` function requires a `WEATHER_API_KEY`. To generate one, please [create an account](https://home.openweathermap.org/users/sign_up). Upon creating an account, select the "API Keys" tab to display your free key.

In [17]:
AV_STOCK_API_KEY = "<AV_STOCK_API_KEY_HERE>"

WEATHER_API_KEY = "<WEATHER_API_KEY_HERE>"

## Step 4. Define the functions

We can now define our functions. In this tutorial, the `get_stock_price` function uses the Stock Market Data API available through Alpha Vantage. 

In [18]:
def get_stock_price(ticker: str, date: str) -> tuple[str, str]:
    """
    Retrieves the lowest and highest stock prices for a given ticker and date.
    Args:
        ticker (str): The stock ticker symbol, e.g., "IBM".
        date (str): The date in "YYYY-MM-DD" format for which you want to get stock prices.
    Returns:
        tuple: A tuple containing the low and high stock prices on the given date, or ("none", "none") if not found.
    """
    print(f"Getting stock price for {ticker} on {date}")
    try:
        stock_url = f"https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={ticker}&apikey={AV_STOCK_API_KEY}"
        stock_data = requests.get(stock_url)
        stock_low = stock_data.json()["Time Series (Daily)"][date]["3. low"]
        stock_high = stock_data.json()["Time Series (Daily)"][date]["2. high"]
        return stock_low, stock_high
    except Exception as e:
        print(f"Error fetching stock data: {e}")
        return "none", "none"

The `get_current_weather` function retrieves the real-time weather in a given location using the Current Weather Data API via [OpenWeather](https://openweathermap.org/api). 

In [19]:
def get_current_weather(location: str) -> dict:
    """
    Fetches the current weather for a given location (default: San Francisco).
    Args:
        location (str): The name of the city for which to retrieve the weather information.
    Returns:
        dict: A dictionary containing weather information such as temperature, weather description, and humidity.
    """
    print(f"Getting current weather for {location}")
    
    try:
        # API request to fetch weather data
        weather_url = f"http://api.openweathermap.org/data/2.5/weather?q={location}&appid={WEATHER_API_KEY}&units=metric"
        weather_data = requests.get(weather_url)
        data = weather_data.json()
        # Extracting relevant weather details
        weather_description = data["weather"][0]["description"]
        temperature = data["main"]["temp"]
        humidity = data["main"]["humidity"]
        
        # Returning weather details
        return {
            "description": weather_description,
            "temperature": temperature,
            "humidity": humidity
        }
    except Exception as e:
        print(f"Error fetching weather data: {e}")
        return {"weather": "NA"}


## Step 5. Set up the API request

Now that our functions are defined, we can create a function that generates a watsonx API request for the provided instructions using the watsonx API endpoint. We will use this function each time we make a request.

In [20]:
def make_api_request(instructions: str) -> dict:
    model_parameters = {
        "decoding_method": "greedy",
        "max_new_tokens": 200,
        "repetition_penalty": 1,
        "stop_sequences": ["\n\n"]
    }
    model = WatsonxLLM(
        model_id=MODEL_ID, 
        url= WATSONX_URL,
        apikey=WATSONX_APIKEY,
        project_id=WATSONX_PROJECT_ID,
        params=model_parameters
    )
    response = model.invoke(instructions)
    return response

Next, we can create a list of available functions. Here, we declare our function definitions that require the function names, descriptions, parameters and required properties. 

In [21]:
functions = [
    {
        "name": "get_current_weather",
        "description": "Get the current weather",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city and country code, e.g. San Francisco, US",
                }
            },
            "required": ["location"],
        },
    },
    {
        "name": "get_stock_price",
        "description": "Retrieves the lowest and highest stock price for a given ticker symbol and date. The ticker symbol must be a valid symbol for a publicly traded company on a major US stock exchange like NYSE or NASDAQ. The tool will return the latest trade price in USD. It should be used when the user asks about the current or most recent price of a specific stock. It will not provide any other information about the stock or company.",
        "parameters": {
            "type": "object",
            "properties": {
                "ticker": {
                    "type": "string",
                    "description": "The stock ticker symbol, e.g. AAPL for Apple Inc.",
                },
                "date": {"type": "string", "description": "Date in YYYY-MM-DD format"},
            },
            "required": ["ticker", "date"],
        },
    },
]

## Step 6. Perform function calling


### Step 6a. Calling the get_stock_price function

To prepare for the API requests, we must set our `query` and a JSON list of the available functions for payload used in the tokenizer chat template.

In [22]:
query = "What were the IBM stock prices on October 7, 2024?"

payload = {
    "functions_str": [json.dumps(x) for x in functions],
}

In [23]:
payload

{'functions_str': ['{"name": "get_current_weather", "description": "Get the current weather", "parameters": {"type": "object", "properties": {"location": {"type": "string", "description": "The city and country code, e.g. San Francisco, US"}}, "required": ["location"]}}',
  '{"name": "get_stock_price", "description": "Retrieves the lowest and highest stock price for a given ticker symbol and date. The ticker symbol must be a valid symbol for a publicly traded company on a major US stock exchange like NYSE or NASDAQ. The tool will return the latest trade price in USD. It should be used when the user asks about the current or most recent price of a specific stock. It will not provide any other information about the stock or company.", "parameters": {"type": "object", "properties": {"ticker": {"type": "string", "description": "The stock ticker symbol, e.g. AAPL for Apple Inc."}, "date": {"type": "string", "description": "Date in YYYY-MM-DD format"}}, "required": ["ticker", "date"]}}']}

Applying a chat template is useful for breaking up long strings of texts into one or more messages with corresponding labels. This allows the LLM to process the input in a format that it expects. Because we want our output to be in a string format, we can set the `tokenize` parameter to false. The `add_generation_prompt` can be set to true in order to append the tokens indicating the beginning of an assistant message to the output. This will be useful when generating chat completions with the model.

In [24]:
chat = [
    {"role":"system","content": f"You are a helpful assistant with access to the following function calls. Your task is to produce a sequence of function calls necessary to generate response to the user utterance. Use the following function calls as required.{payload}"},
    {"role": "user", "content": query }
]

instruction_1 = TOKENIZER.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)
instruction_1

'<|start_of_role|>system<|end_of_role|>You are a helpful assistant with access to the following function calls. Your task is to produce a sequence of function calls necessary to generate response to the user utterance. Use the following function calls as required.{\'functions_str\': [\'{"name": "get_current_weather", "description": "Get the current weather", "parameters": {"type": "object", "properties": {"location": {"type": "string", "description": "The city and country code, e.g. San Francisco, US"}}, "required": ["location"]}}\', \'{"name": "get_stock_price", "description": "Retrieves the lowest and highest stock price for a given ticker symbol and date. The ticker symbol must be a valid symbol for a publicly traded company on a major US stock exchange like NYSE or NASDAQ. The tool will return the latest trade price in USD. It should be used when the user asks about the current or most recent price of a specific stock. It will not provide any other information about the stock or co

Now, we can call the `make_api_request` function and pass the instructions we generated.

In [25]:
data_1 = make_api_request(instruction_1)
data_1

Error getting IAM Token.
Reason: <Response [400]>


WMLClientError: Error getting IAM Token.
Reason: <Response [400]>

The appropriate `get_stock_price` tool use was selected from the set of functions. To run the function, let's extract relevant arguments from the output.

In [26]:
function_call = ast.literal_eval(re.search("({.+})", data_1).group(0))
function_call

NameError: name 'data_1' is not defined

With the function name, ticker and date extracted, we can set these variables and call the function. To call the function using its name as a string, we can use the `globals()` function. 

In [ ]:
function_name = function_call["name"]
ticker = function_call["arguments"]["ticker"]
date = function_call["arguments"]["date"]
stock_info = globals()[function_name](ticker, date)
stock_info

Getting stock price for IBM on 2024-10-07


('225.0200', '227.6700')

The function successfully retrieved the requested stock price. To generate a synthesized final response, we can pass another prompt to the Granite model along with the information collected from function calling. 

In [ ]:
instruction_2 = f"SYSTEM: You are a helpful assistant. Answer this stock information in USD.\n\nUSER: Here is the stock price for {ticker} on {date}:{stock_info}\nASSISTANT:"
data_2 = make_api_request(instruction_2)
data_2

' The stock price for IBM on 2024-10-07 was between 225.02 USD and 227.67 USD.'

### Step 6b. Calling the get_current_weather function

As our next query, let’s inquire about the current weather in San Francisco. We can follow the same steps as in Step 5a by adjusting the query.

In [ ]:
query = "What is the current weather in San Francisco?"

chat = [
    {"role":"system","content": f"You are a helpful assistant with access to the following function calls. Your task is to produce a sequence of function calls necessary to generate response to the user utterance. Use the following function calls as required.{payload}"},
    {"role": "user", "content": query }
]

instruction_1 = TOKENIZER.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)
instruction_1

'<|start_of_role|>system<|end_of_role|>You are a helpful assistant with access to the following function calls. Your task is to produce a sequence of function calls necessary to generate response to the user utterance. Use the following function calls as required.{\'functions_str\': [\'{"name": "get_current_weather", "description": "Get the current weather", "parameters": {"type": "object", "properties": {"location": {"type": "string", "description": "The city and country code, e.g. San Francisco, US"}}, "required": ["location"]}}\', \'{"name": "get_stock_price", "description": "Retrieves the lowest and highest stock price for a given ticker symbol and date. The ticker symbol must be a valid symbol for a publicly traded company on a major US stock exchange like NYSE or NASDAQ. The tool will return the latest trade price in USD. It should be used when the user asks about the current or most recent price of a specific stock. It will not provide any other information about the stock or co

In [ ]:
data_1 = make_api_request(instruction_1)
data_1

"{'function_calls': [{'name': 'get_current_weather', 'arguments': {'location': 'San Francisco, US'}}]}"

In [ ]:
function_call = ast.literal_eval(re.search("({.+})", data_1).group(0))
function_call

{'function_calls': [{'name': 'get_current_weather',
   'arguments': {'location': 'San Francisco, US'}}]}

Once again, the model decides the appropriate function, in this case `get_current_weather`, and extracts the location correctly. Now, let's call the function with the argument generated by the model. 

In [ ]:
function_name = function_call['function_calls'][0]['name']
location = function_call['function_calls'][0]["arguments"]["location"]
weather_info = globals()[function_name](location)
weather_info

Getting current weather for San Francisco, US


{'description': 'few clouds', 'temperature': 16.97, 'humidity': 43}

The function response correctly describes the current weather in San Francisco. Lastly, let's generate a synthesized final response with the results of this function call. 

In [ ]:
instruction_2 = f"SYSTEM: You are a helpful assistant. Synthesize this weather information.\n\nUSER: Here is the current weather in {location}: {weather_info}\nASSISTANT:"
data_2 = make_api_request(instruction_2)
data_2

' The current weather in San Francisco, US is few clouds, with a temperature of 16.97 and humidity of 43.'

## Summary

In this tutorial, you built custom functions and used the Granite-3.0-8B-Instruct model to determine which function to call based on  key information from user queries. With this information, you called the function with the arguments as stated in the model response. These function calls produce the expected output. Finally, you called the Granite-3.0-8B-Instruct model again to synthesize the information returned by the functions. 